In [1]:
import torch
import string
import random
import re
import time, math

import torch
import torch.nn as nn
from torch.autograd import Variable

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [2]:
# Ler um arquivo em texto puro contendo o conteúdo a ser aprendido
file = open('lfv_texts_plain.txt').read()
file_len = len(file)
print('file_len =', file_len)

all_characters = ''.join(set(file))
n_characters = len(all_characters)

print(all_characters, n_characters)

file_len = 1298370
ô",u va”zK/…íF6HÀx5õú(óo‘niw@—
è̀RV–-X’̂rc*bD jqÕ$dt~T9OQîI°h7É3)C0YfN!ÇàêJÿ%l1Um;4Aáe?üâ8GW`PÃBé&́ZELsç̃ÁgM―̧p.'Sk:Ó2ã“ìÍ 123


In [3]:
chunk_len = 200

# Gera um pedaço aleatório de texto com o tamanho especificado em chuck_len
def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())

a Andradina. O Bolota só ficou de olhos arregalados mas o resto da família jurou não dizer nada. Fora da casa, ninguém precisava saber que a Andradina estava voltando.
A chegada da Andradina só na


In [4]:
# Converte string para uma lista de inteiros

def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
      try:
        tensor[c] = all_characters.index(string[c])
      except:
        print(c)
        raise
    return Variable(tensor)

print(char_tensor('abcDEF'))

tensor([  6,  43,  41,  44, 101,  13])


In [13]:
# Gera um 'exemplo' aleatório (um pedaço aleatório convertido para lista de inteiros)
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

In [14]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        input = self.encoder(input.view(1, -1))
        output, hidden = self.gru(input.view(1, 1, -1), hidden)
        decoded_output = self.decoder(output.view(1, -1))
        return decoded_output, hidden

    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))

In [20]:
# Gera um texto iniciando com "prime_str" de tamanho "predict_len" e variação definida por "temperature"
def evaluate(prime_str='A', predict_len=100, temperature=0.8):
    hidden = rnn.init_hidden()
    prime_input = char_tensor(prime_str)
    predicted = prime_str

    # prime_str é o texto inicial que o gerado irá completar
    for p in range(len(prime_str) - 1):
        _, hidden = rnn(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = rnn(inp, hidden)
        
        # Usa a temperatura para amostrar a distribuição e escolher a saída probabilísticamente
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Adiciona o caracter predito à string de saída
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = char_tensor(predicted_char)

    return predicted

In [21]:
# Treina sobre um exemplo (i.e. uma amostragem do texto)

def train(inp, target):
    hidden = rnn.init_hidden()
    rnn.zero_grad()
    loss = 0

    inp = inp
    hidden = hidden
    for c in range(chunk_len):
        output, hidden = rnn(inp[c], hidden)
        loss += loss_metric(output, target[c].unsqueeze(0))

    loss.backward()
    optimizer.step()

    return loss.data.item() / chunk_len

In [ ]:
n_epochs = 50000
print_every = 100
hidden_size = 512
n_layers = 2
lr = 0.0005

rnn = RNN(n_characters, hidden_size, n_characters, n_layers)

optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)
loss_metric = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

for epoch in range(1, n_epochs + 1):
    example = random_training_set()
    loss = train(example[0],example[1])       
    loss_avg += loss

    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
        print(evaluate('Os', 100), '\n')


 44s (100 0%) 2.3991]
Os procalina dadio eriste contade. verso a de de de mosum o ghaco hom nas tanhacte çara, Nãoçavente 

 19s (200 0%) 2.3551]
Os exas mande Vimora mulhadar mara homo, nrocer mara prata Parreta a pado que doconta, prara um shoi m 

 52s (300 0%) 2.0358]
Os peló Bé consos precia seve. Taixa forviz. Fivos estencias a cúriam acambraz, em comlivar cênil c 

 25s (400 0%) 1.9768]
Os do Beramende com o Astido soapos ainda. Destoio aquera. Ela não a gidão. O a mente e secentas, pa 

 56s (500 1%) 1.8918]
Os agrante e adais e diz pessor de algumpde lebal, Vegunte no o Homem semente, eu tempo de Calimo, a a 

 26s (600 1%) 1.9929]
Os do prate, não tamos paração é a minhaçãos e toberam o precite exitar. Í....
― Partas ainte.  

 56s (700 1%) 1.9931]
Os nunca concusas de explicous contente.
Porto a Mortos pois coma do bontícico. Com conssssientas des 



In [ ]:
# Com a rede treinada, gerar um texto longo

print(evaluate('O psicanalista de', 5500), '\n')